In [1]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('max_rows', 200)
pd.set_option('max_columns', 40)

import networkx as nx,community
import matplotlib.pyplot as plt
plt.style.use('ggplot')
font = {'family' : 'sans',
        'weight' : 'bold',
        'size'   : 20}
plt.rc('font', **font)

plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams[u'figure.figsize'] = (16,8)

In [2]:
#mas librerias que voy obteniendo
import biopandas.pdb as bp
biop = bp.PandasPdb() #libreria de lectura de pdbs
#libreria de calculo de distancia euclidiana
from scipy.spatial.distance import pdist, squareform
#libreria de mate
import numpy as np
import itertools as it

In [3]:
# generar mismos resultados que el algoritmo click
path1 ='/Users/serch/pdbmani/Serch/1phr.pdb'
path2 ='/Users/serch/pdbmani/Serch/1tig.pdb'

#funcion de lectura con biopandas
def read_biopdb(path):
        df = biop.read_pdb(path)
        df_atom = df.df['ATOM']
        #OJO AQUI ESTA ADECUADO AL PDB   para elegir solo un frame en trj_0 y trj_0_A [:1805]
        df_ca = df_atom[df_atom.atom_name == 'CA'][[
        'atom_number','atom_name','residue_name','residue_number',
        'x_coord','y_coord','z_coord']]
        columna_vector = []
        for i in zip(df_ca.x_coord.tolist(),df_ca.y_coord.tolist(),df_ca.z_coord.tolist()):
            columna_vector.append(list(i))
            
        df_ca['vector'] = columna_vector
        return(df_ca)

In [4]:
df_ca1 = read_biopdb(path1)
df_ca2 = read_biopdb(path2)

In [6]:
#se calcula la distancia entre cada par de nodos.
def distancia_entre_atomos(df_ca):
    distancias = []
    #se calcula la distancia euclidiana entre cada atomo de carbon alfalfa
    for v,i in zip(df_ca.vector,df_ca.atom_number):
        distancia_un_atomo = []
        for av,j in zip(df_ca.vector,df_ca.atom_number):
            distancia = pdist([v,av],metric='euclidean').item()
            distancia_un_atomo.append(distancia)
        distancias.append(distancia_un_atomo)
    #se genera la matriz de adyacencias para la red
    df_da = pd.DataFrame(index=df_ca.atom_number,columns=df_ca.atom_number,data=distancias)
    return(df_da)

In [7]:
df_da1 = distancia_entre_atomos(df_ca1)
df_da2 = distancia_entre_atomos(df_ca2)

In [10]:
def gen_3_cliques(df_da, dth = 10, k=3):
    """Genera n-cliques de dataframe de distancias, tomando en cuenta los enlaces menores o iguales
    a dth y forma los k-cliques que elijas 
    valores por default:
    dth=10, k=3"""
    #red de distancias completa
    red = nx.from_pandas_adjacency(df_da)
    print("red antes de filtros:",nx.info(red))

    #filtro de distancias
    edgesstrong = [(u,v) for (u,v,d) in red.edges(data=True) if d["weight"] <= dth]

    red = nx.Graph(edgesstrong)
    print("=="*20)
    print("red despues de filtros:",nx.info(red))

    n_cliques = [clq for clq in nx.find_cliques(red) if len(clq) >=k]
    print('numero de cliques maximos encontrados:',len(n_cliques))

    lista_cliques = []
    for i,v in enumerate(n_cliques):
        a = list(it.combinations(v,k))
        for j in a:
            if set(j) not in lista_cliques:
                #recuerda que para comparar elementos utiliza set, y apilalos como set
                lista_cliques.append(set(j))

    df_lc = pd.DataFrame(lista_cliques)            
    print("numero de %s-cliques posibles:" % (k), df_lc.shape[0])
    return(df_lc)

In [12]:
df_lc1 = gen_3_cliques(df_da1)
print('--'*50)
df_lc2 = gen_3_cliques(df_da2)

red antes de filtros: Name: 
Type: Graph
Number of nodes: 154
Number of edges: 11781
Average degree: 153.0000
red despues de filtros: Name: 
Type: Graph
Number of nodes: 154
Number of edges: 1378
Average degree:  17.8961
numero de cliques maximos encontrados: 419
numero de 3-cliques posibles: 4480
----------------------------------------------------------------------------------------------------
red antes de filtros: Name: 
Type: Graph
Number of nodes: 88
Number of edges: 3828
Average degree:  87.0000
red despues de filtros: Name: 
Type: Graph
Number of nodes: 88
Number of edges: 709
Average degree:  16.1136
numero de cliques maximos encontrados: 246
numero de 3-cliques posibles: 2102


In [13]:
df_lc1.head()

,0,1,2
0,121,514,431
1,121,514,507
2,121,514,393
3,121,514,387
4,514,507,431
5,393,514,431
6,514,387,431
7,393,514,507
8,514,507,387
9,393,514,387
